In [1]:
from result_saver import SaverProvider
provider = SaverProvider()

In [8]:
import numpy as np
# parameter grid
distances = np.arange(5, 57, 2)
distances = distances[::-1]
rounds = 35 # take the biggest that is possible
devices = ["ibm_nazca", "ibm_sherbrooke", "ibm_brisbane", "ibm_cusco"]
logicals = ['0', '1']

print("Distances: ", distances)

Distances:  [55 53 51 49 47 45 43 41 39 37 35 33 31 29 27 25 23 21 19 17 15 13 11  9
  7  5]


# Sligthly faster faster triple loop

In [9]:
from tqdm import tqdm

from qiskit_qec.circuits import RepetitionCodeCircuit
from qiskit import transpile

from soft_info import find_longest_path_in_hex
from Scratch import metadata_helper


hex_backend = provider.get_backend('ibm_sherbrooke')
path, _, _  = find_longest_path_in_hex(hex_backend)

for distance in tqdm(distances):  
    bounded_path = path[:2 * distance - 1]
    layout = bounded_path[1::2] + bounded_path[::2] 
    code = RepetitionCodeCircuit(distance, rounds) 
    for device in devices:
        backend = provider.get_backend(device)
        for logical in logicals:          
            qc = code.circuit[logical]                                   
            transpiled_qc = transpile(qc, backend, initial_layout=layout) #, optimization_level=3)

            n_shots = int(1e6 / (distance*rounds))
            metadata = metadata_helper(descr='diff rounds v2', code="RepetitionCodeCircuit", distance=int(distance), rounds=f"{rounds}",  logical=logical, layout='_is_hex=True, sherbrooke')
            backend.run(metadata, transpiled_qc, shots=int(n_shots), meas_level=1, meas_return='single', job_tags=["RepetitionCodeCircuit"]) # meas_level=1 for IQ  
            


  0%|          | 0/26 [00:00<?, ?it/s]

19:15:43 Warning: updating metadata. IN FILE: /Users/mha/My_Drive/Desktop/Studium/Physik/MSc/Semester_3/IBM/IBM_GIT/Soft-Info/libs/qiskit-saver-provider/result_saver/provider/saver_provider.py, LINE: 46


100%|██████████| 26/26 [21:11<00:00, 48.90s/it]
